# Hi-C pipeline
*Author: Qin CAO*
## Raw .fastq files to Hi-C contact matrices
### *[Juicer] (https://github.com/aidenlab/juicer/wiki)*

Juicer is a very powerful tool that deals with Hi-C data. It can map .fastq files to .hic files, which is a binary format efficiently stores contact matrices at multiple resolutions. 
#### [Installation of Juicer] (https://github.com/aidenlab/juicer/wiki/Installation)
#### Configuration of juicer.sh file
We need to modify the configurations in juicer.sh file based on our own data. Some configurations can also be modified by passing the parameters when running juicer.sh. The key points are listed below. 


In [ ]:
read1str="_R1" #suffix of paired read
read2str="_R2" #suffix of paired read
site="MboI" #enzyme site
genomeID="hg19" #genome ID
ligation="GATCGATC" #enzyme site ligation junction sequence

#### An example of processing raw .fastq files to .hic files

In [ ]:
./juicer.sh -g mm10 -s MboI 
#In this example, the reference genome is mm10 and the enzyme is MboI. 

#### QCs and basic statistics
Juicer provides QCs and basic statistics. We summarize some key definitions/terms here. 
* Sequenced Read Pairs=Normal Paired+Chimeric Paired+Chimeric Ambiguous+Unmapped
* Alignable=Normal Paired+Chimeric Paired
* Alignable=Unique Reads+PCR Duplicates+Optical Duplicates
* Unique Reads=Intra-fragment Reads+Below MAPQ Threshold+Hi-C Contacts
* Hi-C Contacts=Inter-chromosomal+Intra-chromosomal

### *[HiC-Pro](https://github.com/nservant/HiC-Pro)(alternative tool)*
We would suggest HiC-Pro as an alternative tool, which is excellent in summarizing QCs. Juicer also provides QCs but sometimes HiC-Pro is more informative. However, HiC-Pro does not provide .hic files directly, which limits its convenience in standard downstream analyses. 

## Extract interactions from .hic files
### *[Juicer dump](https://github.com/aidenlab/juicer/wiki/Data-Extraction)*
The parameters of juicer dump include the kind of matrix, normalization method, chromosome coordinates and bin resolution.
#### A usage example of dump

In [ ]:
java -jar $juicer_tools dump observed KR inter.hic 4 4 BP 50000 4.txt
#This will dump the observed intrachromosomal matrix of chromosome 4 with KR normalization at 50Kb resolution to the file 4.txt.

#### The example output of the example above

In [ ]:
#bin1_start bin2_start KR_normalized_reads
3050000 3050000 689.73926
3050000 3100000 39.28981
3100000 3100000 558.02704
3050000 3150000 20.03267
3100000 3150000 71.13022
3150000 3150000 470.02182
3050000 3200000 10.919163
3100000 3200000 27.693422
3150000 3200000 47.443287
3200000 3200000 457.82416

## Call A/B compartment
### *[Juicer eigenvector](https://github.com/aidenlab/juicer/wiki/Eigenvector)*
It computes the first principal component(PC1) of the Hi-C contact matrix, in which the sign indicates the compartment(e.g. + indicates compartment A and - indicates compartment B or vise versa). A note here is that it is hard to compute eigenvectors for a very sparse matrix. Typically it can only handle relatively lower resolutions(e.g. 500kb), which is also consistent with the definition of A/B compartments. 
#### A usage example of eigenvector

In [ ]:
java -jar juicer_tools.jar eigenvector VC HIC001.hic X BP 5000 eigen.txt
#This will calculate the eigenvector of chromosome X with VC normalization at 5Kb resolution and print to eigen.txt.

#### An example output

In [ ]:
#Each line is a bin with PC1 value
#chr    bin_start bin_end PC1 
chr10   3500000 4000000 -0.00357968
chr10   4000000 4500000 -0.0702789
chr10   4500000 5000000 -0.00730329
chr10   5000000 5500000 -0.00533751
chr10   5500000 6000000 -0.00196586
chr10   6000000 6500000 0.0402201

## Call TAD
### *[Hi-C Domain Caller](http://chromosome.sdsc.edu/mouse/hi-c/download.html)*
We perfer Hi-C Domain Caller to *[Juicer arrowhead](https://github.com/aidenlab/juicer/wiki/Arrowhead)* for two reasons: 1) arrowhead is designed for finding contact domains rather than TADs specifically; 2) it is too hard to set reasonable thresholds in arrowhead for finding TADs.  

#### An example output

In [ ]:
#Each line is a TAD
#chr    TAD_start       TAD_end
chr10   11350000        13000000 
chr10   13000000        14400000 
chr10   14750000        17750000 
chr10   18900000        20350000 
chr10   20350000        20950000

## Call significant Hi-C interactions
### *[Fit-Hi-C](https://bioconductor.org/packages/release/bioc/vignettes/FitHiC/inst/doc/fithic.html)*
A q-value will be assigned to each Hi-C interaction by Fit-Hi-C to indicate the interaction significance. Fit-Hi-C is actively updated in [R version](https://bioconductor.org/packages/release/bioc/vignettes/FitHiC/inst/doc/fithic.html) but not [python version](https://github.com/ay-lab/fithic). So we recommend to use R version. 
#### An example output

In [ ]:
#chr_1  bin1_mid        chr_2   bin2_mid        contact         pvalue  qvalue
chr1    fragmentMid1    chr2    fragmentMid2    contactCount    p_value q_value
chr2    25000   chr2    75000   4245    0.0140615961205994      0.132152059562782

## Data visualization
### *[Juicebox](https://github.com/aidenlab/Juicebox)*
Juicebox can load .hic files and display Hi-C contact matrices. The web version of Juicebox is [here](https://www.aidenlab.org/juicebox/). 
#### An example image
<!--![Image juicebox](https://lh3.googleusercontent.com/-GbPE2TrUzfk/V8h_w_wHHzI/AAAAAAAAdQk/0XEwOE-Z0mQs7YV4MxG-RfO5hgzt-aGMQCLcB/s1600/neva%2Bemail.jpg)]-->
<img src="https://lh3.googleusercontent.com/-GbPE2TrUzfk/V8h_w_wHHzI/AAAAAAAAdQk/0XEwOE-Z0mQs7YV4MxG-RfO5hgzt-aGMQCLcB/s1600/neva%2Bemail.jpg" width="800" height="400" />

### *[UCSC session](https://genome.ucsc.edu/cgi-bin/hgSession?hgS_doMainPage=1)(mainly for A/B compartments and TADs)*
The advantage of UCSC session is that other signal tracks(e.g. histone modifications) could also be loaded at the same time together with A/B compartments and TADs.

### *[3D Genome Browser](http://promoter.bx.psu.edu/hi-c/)*
<!--![3D Genome Browser example](http://promoter.bx.psu.edu/hi-c/img/capture-HiC.jpg)-->
<img src="http://promoter.bx.psu.edu/hi-c/img/capture-HiC.jpg" width="800" height="400"/>


### *[WashU Epigenome Browser](https://epigenomegateway.wustl.edu/)*
<!--![WashU Epigenome Browser example](https://portal.biohpc.swmed.edu/media/filer_public_thumbnails/filer_public/78/6d/786d31a1-bb99-49a3-ad1f-635244519929/7epi_result.png__800x378_q85_crop_subsampling-2_upscale.png)-->

<img src="https://portal.biohpc.swmed.edu/media/filer_public_thumbnails/filer_public/78/6d/786d31a1-bb99-49a3-ad1f-635244519929/7epi_result.png__800x378_q85_crop_subsampling-2_upscale.png" width="800" height="400"/>